In [30]:
# Dependencies
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pandas as pd
import pymongo

In [5]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [6]:
# Define database and collection
db = client.commerce_db
collection = db.items

In [7]:
# Scrape page
url = 'https://mars.nasa.gov/news/'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [8]:
# Examine the results, then determine element that contains sought info
# results are returned as an iterable list
title_results = soup.find_all('div', class_='slide')[0]

#Print scraped article header and description
title_results

<div class="slide">
<div class="image_and_description_container">
<a href="/news/8896/nasa-ingenuity-mars-helicopter-prepares-for-first-flight/">
<div class="rollover_description">
<div class="rollover_description_inner">
Now uncocooned from its protective carbon-fiber shield, the helicopter is being readied for its next steps.  
</div>
<div class="overlay_arrow">
<img alt="More" src="/assets/overlay-arrow.png"/>
</div>
</div>
<img alt="NASA Ingenuity Mars Helicopter Prepares for First Flight" class="img-lazy" data-lazy="/system/news_items/list_view_images/8896_thumb-320.jpg" src="/assets/loading_320x240.png"/>
</a>
</div>
<div class="content_title">
<a href="/news/8896/nasa-ingenuity-mars-helicopter-prepares-for-first-flight/">
NASA Ingenuity Mars Helicopter Prepares for First Flight
</a>
</div>
</div>

In [9]:
#Define title and content
article_title = "NASA Ingenuity Mars Helicopter Prepares for First Flight"
paragraph_text = "Now uncocooned from its protective carbon-fiber shield, the helicopter is being readied for its next steps."

In [10]:
#Pull featured image using splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\Captain\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [11]:
url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(url)

In [23]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
mars_image = soup.find_all('a', class_='showimg fancybox-thumbs')
mars_image

[<a class="showimg fancybox-thumbs" href="image/featured/mars2.jpg" target="_blank"> <button class="btn btn-outline-light"> FULL IMAGE</button></a>]

In [13]:
#Add index.html base to href data to create full image link
featured_image = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars2.jpg'

In [49]:
#Pull table from Mars Facts page.
mars_page = pd.read_html('https://space-facts.com/mars/')[0]

In [50]:
mars_page

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers
